# Demand Forecast Model

This notebook generates demand forecasts and inventory projections by SKU and planning category.

Setup and Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

Configuration

- Set file paths and parameters here.

In [2]:
import os

# Get the directory where this notebook is located
base_path = os.path.dirname(os.path.abspath('__file__'))
# Repository root is one level up from notebooks folder
repo_root = os.path.join(base_path, '..')

# File paths (relative to repository root)
catalog_file = os.path.join(repo_root, 'catalog_2025-12-09-1340.csv')
inventory_file = os.path.join(repo_root, 'on hand inventory_2025-12-09-1341.csv')
sales_file = os.path.join(repo_root, 'sku sales_2025-12-09-1347.csv')
on_order_file = os.path.join(repo_root, 'CZ On Order Sample Data.xlsx')
output_file = os.path.join(repo_root, 'Demand_Forecast_Inventory_Model.xlsx')

# Output file paths (save to data folder)
data_path = os.path.join(repo_root, 'data')
os.makedirs(data_path, exist_ok=True)  # Create data folder if it doesn't exist
demand_forecast_file = os.path.join(data_path, 'demand_forecast_file.csv')
demand_forecast_sku_file = os.path.join(data_path, 'demand_forecast_sku.csv')
demand_forecast_pivot_file = os.path.join(data_path, 'demand_forecast_pivot.xlsx')

# ROS and Curve data files
ros_file = os.path.join(repo_root, 'CZ Sample ROS Data.csv')
curve_file = os.path.join(repo_root, 'CZ Sample Curve data.xlsx')

Load Data Files

In [3]:
print("Loading data files...")

# Load catalog
catalog = pd.read_csv(catalog_file)
catalog = catalog.rename(columns={'SKU': 'SKU'})

# Load sales
sales = pd.read_csv(sales_file)
sales = sales.rename(columns={'COMPONENT_SKU': 'SKU'})
sales['ORDER_DATE'] = pd.to_datetime(sales['ORDER_DATE'])
sales['ORDER_MONTH'] = pd.to_datetime(sales['ORDER_MONTH'])

# Load ROS data (daily rate of sale)
ros_data = pd.read_csv(ros_file)
ros_data.columns = ros_data.columns.str.strip()
ros_data['VARIANT_SKU'] = ros_data['VARIANT_SKU'].str.strip()
# Convert ROS to numeric, treating '-' and blanks as 0
ros_data['NORMALIZED_ROS'] = pd.to_numeric(ros_data['NORMALIZED_ROS'], errors='coerce').fillna(0)
# Create a lookup dictionary for ROS by SKU
ros_lookup = dict(zip(ros_data['VARIANT_SKU'], ros_data['NORMALIZED_ROS']))
print(f"Loaded ROS data for {len(ros_lookup)} SKUs")

# Load Curve data (monthly sales curve by category)
curve_data = pd.read_excel(curve_file)
# Column B is the category (matches CATEGORY field in catalog)
curve_data = curve_data.rename(columns={'Gross Item Finance Forecast CURVE': 'CATEGORY'})
# Set category as index for easy lookup
curve_data = curve_data.set_index('CATEGORY')
# Drop any unnamed columns (na=False treats NaN column names as False, not NaN)
curve_data = curve_data.loc[:, ~curve_data.columns.str.contains('Unnamed', na=False)]
# Convert column names to datetime for easier matching
curve_data.columns = pd.to_datetime(curve_data.columns, errors='coerce')
print(f"Loaded Curve data for {len(curve_data)} categories: {curve_data.index.tolist()}")

Loading data files...


Loaded ROS data for 4421 SKUs


Loaded Curve data for 10 categories: ['ACCENTS', 'BASKETS', 'BATH', 'BEDDING', 'BLANKETS', 'FURNITURE', 'FURNITURE - MTO', 'PILLOWS', 'RUGS', 'TABLEWARE']


Define Time Horizons

In [4]:
# Determine the most recent date with actual sales data
max_sales_date = sales['ORDER_DATE'].max()
print(f"Most recent date with actual sales data: {max_sales_date.strftime('%Y-%m-%d')}")

# Determine current date and forecast horizon
current_date = datetime(2025, 12, 1)
history_start = datetime(2023, 1, 1)
forecast_end = datetime(2026, 12, 31)  # Forecast through end of 2026

# Create month range for historical + forecast
all_months = pd.date_range(start=history_start, end=forecast_end, freq='MS')
historical_months = [m for m in all_months if m <= current_date]
forecast_months_list = [m for m in all_months if m > current_date]

print(f"Historical months: {len(historical_months)}, Forecast months: {len(forecast_months_list)}")

# Load current-month daily forecast to measure progress
progress_file = os.path.join(repo_root, 'data', 'Current Month Daily Forecast.csv')
col_forecast = 'Discounted Rev Forecast'  # Column B
col_actual = 'Actual Discounted Revenue'  # Column C
pct_month_complete = np.nan
pct_month_left = np.nan

if os.path.exists(progress_file):
    progress_df = pd.read_csv(progress_file)
    missing_cols = [c for c in (col_forecast, col_actual) if c not in progress_df.columns]
    if missing_cols:
        print(f"Warning: {progress_file} missing columns: {missing_cols}")
    else:
        # Helper function to clean currency values (remove $, commas, and whitespace)
        def clean_currency(value):
            if pd.isna(value) or value == '':
                return 0
            if isinstance(value, str):
                return float(value.replace('$', '').replace(',', '').strip())
            return float(value)
        
        # Convert currency strings to numeric values
        actual_series = progress_df[col_actual].apply(clean_currency)
        forecast_series = progress_df[col_forecast].apply(clean_currency)

        # Find last row with a filled actual value (not NaN and > 0)
        last_filled_idx = actual_series[actual_series.notna() & (actual_series > 0)].last_valid_index()

        if last_filled_idx is not None:
            actuals_to_date = actual_series.loc[:last_filled_idx].sum()
            remaining_forecast = forecast_series.loc[last_filled_idx + 1:].sum()
        else:
            # No actuals yet; everything is remaining
            actuals_to_date = 0
            remaining_forecast = forecast_series.sum()

        denom = actuals_to_date + remaining_forecast
        pct_month_complete = actuals_to_date / denom if denom else np.nan
        pct_month_left = 1 - pct_month_complete if denom else np.nan

        print(f"Current-month progress loaded from: {progress_file}")
        print(f"  Actuals to date: ${actuals_to_date:,.2f}")
        print(f"  Remaining forecast: ${remaining_forecast:,.2f}")
        print(f"  Total: ${denom:,.2f}")
        print(f"  % month complete: {pct_month_complete:.2%}" if not np.isnan(pct_month_complete) else "  % month complete: NaN")
        print(f"  % month left: {pct_month_left:.2%}" if not np.isnan(pct_month_left) else "  % month left: NaN")
else:
    print(f"Warning: {progress_file} not found; pct_month_complete remains NaN")

Most recent date with actual sales data: 2025-12-09
Historical months: 36, Forecast months: 12
Current-month progress loaded from: C:\Users\JaeheeKim\demand-forecast-hackathon-1\notebooks\..\data\Current Month Daily Forecast.csv
  Actuals to date: $1,008,933.00
  Remaining forecast: $1,360,000.00
  Total: $2,368,933.00
  % month complete: 42.59%
  % month left: 57.41%


Aggregate Sales Data

In [5]:
# Aggregate sales by SKU and month
sales_agg = sales.groupby(['SKU', 'ORDER_MONTH'])['UNITS_SOLD'].sum().reset_index()
sales_agg = sales_agg.rename(columns={'ORDER_MONTH': 'MONTH'})


# Get all unique SKUs from catalog
all_skus = catalog['SKU'].unique()
print(f"Total unique SKUs: {len(all_skus)}")

Total unique SKUs: 6109


Define Forecast Function

In [6]:
def calculate_forecast(sku, category, forecast_month, ros_lookup, curve_data, daily_ros=None, annual_units=None):
    """
    Calculate forecast for a SKU using:
    1. ROS (Rate of Sale) data - daily ROS x 365 days
    2. Monthly sales curve applied by category to annual units

    Args:
        sku: The SKU identifier
        category: The category for curve lookup (from catalog CATEGORY field)
        forecast_month: The month to forecast (datetime)
        ros_lookup: Dictionary of SKU -> daily ROS
        curve_data: DataFrame with monthly curve percentages by category
        daily_ros: Optional pre-calculated daily ROS (if None, will calculate from ros_lookup)
        annual_units: Optional pre-calculated annual units (if None, will calculate from daily_ros)

    Returns:
        Monthly forecasted units
    """
    # Get daily ROS for this SKU (use provided value or calculate)
    if daily_ros is None:
        daily_ros = ros_lookup.get(sku, 0)

    # Calculate annual units from daily ROS (daily × 365 days) (use provided value or calculate)
    if annual_units is None:
        annual_units = daily_ros * 365

    # Get the monthly curve percentage for this month and category
    # The curve represents the proportion of annual sales for each month
    month_curve_pct = 1.0 / 12.0  # Default to average (1/12) if no curve found

    # Use category directly from catalog (no mapping needed)
    if category and category in curve_data.index:
        # Find the matching month in the curve data (match by month number)
        forecast_month_num = forecast_month.month
        for curve_col in curve_data.columns:
            if pd.notna(curve_col) and curve_col.month == forecast_month_num:
                # Get the curve percentage for this month
                month_curve_pct = curve_data.loc[category, curve_col]
                break

    # Calculate monthly forecast: annual units × monthly curve percentage
    unit_demand = annual_units * month_curve_pct

    return unit_demand

Generate Forecast Data Frame

In [7]:
## Generate Forecast 

# Create a dataframe with all SKU and month combinations
forecast_output = pd.DataFrame({
    'SKU': np.repeat(all_skus, len(all_months)),
    'MONTH': np.tile(all_months, len(all_skus))
})

# Add daily ROS
forecast_output['daily_ros'] = forecast_output['SKU'].map(ros_lookup).fillna(0)

# Calculate annual units (daily ROS × 365)
forecast_output['annual_units'] = forecast_output['daily_ros'] * 365

# Get category for each SKU from catalog
sku_category_map = dict(zip(catalog['SKU'], catalog['CATEGORY']))
forecast_output['CATEGORY'] = forecast_output['SKU'].map(sku_category_map)

# Calculate unit_demand using the forecast function (pass pre-calculated values to avoid duplication)
print("Calculating forecasts for all SKU-month combinations...")
forecast_output['unit_demand'] = forecast_output.apply(
    lambda row: calculate_forecast(
        row['SKU'], 
        row['CATEGORY'], 
        row['MONTH'], 
        ros_lookup, 
        curve_data,
        daily_ros=row['daily_ros'],  # Pass pre-calculated value
        annual_units=row['annual_units']  # Pass pre-calculated value
    ), 
    axis=1
)

# Merge units_sold from sales_agg (will be null/zero for future months)
forecast_output = forecast_output.merge(
    sales_agg[['SKU', 'MONTH', 'UNITS_SOLD']], 
    on=['SKU', 'MONTH'], 
    how='left'
)
# Set units_sold: use UNITS_SOLD for historical months, NaN for future months
forecast_output['units_sold'] = forecast_output.apply(
    lambda row: row['UNITS_SOLD'] if row['MONTH'] <= current_date else np.nan,
    axis=1
)
# Fill NaN for historical months that don't have sales data with 0
forecast_output.loc[forecast_output['MONTH'] <= current_date, 'units_sold'] = forecast_output.loc[forecast_output['MONTH'] <= current_date, 'units_sold'].fillna(0)

# Select and reorder final columns
forecast_output = forecast_output[['SKU', 'MONTH', 'daily_ros', 'annual_units', 'unit_demand', 'units_sold']]

# Sort by SKU and MONTH
forecast_output = forecast_output.sort_values(['SKU', 'MONTH']).reset_index(drop=True)

print(f"\nForecast output created:")
print(f"  Total rows: {len(forecast_output):,}")
print(f"  Unique SKUs: {forecast_output['SKU'].nunique():,}")
print(f"  Date range: {forecast_output['MONTH'].min()} to {forecast_output['MONTH'].max()}")
print(f"\nSample output:")
print(forecast_output.head(10))


Calculating forecasts for all SKU-month combinations...



Forecast output created:
  Total rows: 293,232
  Unique SKUs: 6,109
  Date range: 2023-01-01 00:00:00 to 2026-12-01 00:00:00

Sample output:
                         SKU      MONTH  daily_ros  annual_units  unit_demand  \
0  000004556-004-FL-S-002-B1 2023-01-01        0.0           0.0          0.0   
1  000004556-004-FL-S-002-B1 2023-02-01        0.0           0.0          0.0   
2  000004556-004-FL-S-002-B1 2023-03-01        0.0           0.0          0.0   
3  000004556-004-FL-S-002-B1 2023-04-01        0.0           0.0          0.0   
4  000004556-004-FL-S-002-B1 2023-05-01        0.0           0.0          0.0   
5  000004556-004-FL-S-002-B1 2023-06-01        0.0           0.0          0.0   
6  000004556-004-FL-S-002-B1 2023-07-01        0.0           0.0          0.0   
7  000004556-004-FL-S-002-B1 2023-08-01        0.0           0.0          0.0   
8  000004556-004-FL-S-002-B1 2023-09-01        0.0           0.0          0.0   
9  000004556-004-FL-S-002-B1 2023-10-01        0

Create Output Files
- for planner review if desired

In [8]:
# 1. demand_forecast_file: SKU, MONTH, UNIT DEMAND, UNIT SALES
demand_forecast_output = forecast_output[['SKU', 'MONTH', 'unit_demand', 'units_sold']].copy()
demand_forecast_output = demand_forecast_output.rename(columns={
    'SKU': 'SKU',
    'MONTH': 'MONTH',
    'unit_demand': 'UNIT DEMAND',
    'units_sold': 'UNIT SALES'
})
demand_forecast_output.to_csv(demand_forecast_file, index=False)
print(f"✓ Created demand_forecast_file: {demand_forecast_file}")
print(f"  Rows: {len(demand_forecast_output):,}")

# 2. demand_forecast_sku: SKU, DAILY ROS, ANNUAL UNITS (one row per SKU)
demand_forecast_sku = forecast_output[['SKU', 'daily_ros', 'annual_units']].drop_duplicates(subset=['SKU']).copy()
demand_forecast_sku = demand_forecast_sku.rename(columns={
    'SKU': 'SKU',
    'daily_ros': 'DAILY ROS',
    'annual_units': 'ANNUAL UNITS'
})
demand_forecast_sku = demand_forecast_sku.sort_values('SKU').reset_index(drop=True)
demand_forecast_sku.to_csv(demand_forecast_sku_file, index=False)
print(f"\n✓ Created demand_forecast_sku: {demand_forecast_sku_file}")
print(f"  Unique SKUs: {len(demand_forecast_sku):,}")

# 3. Excel file with SKU, METRIC, catalog fields, and monthly columns
# Each SKU will have 2 rows: one for Unit Sales, one for Unit Demand

# Get catalog fields for each SKU
required_catalog_fields = ['SKU', 'SKU_DESCRIPTION', 'CATEGORY', 'SUB_CATEGORY', 'COLLECTION', 
                           'PLANNING_STATUS', 'LAUNCH_YEAR_SEASON', 'VENDOR_COST_USD', 'FULL_PRICE_RETAIL']
available_fields = [field for field in required_catalog_fields if field in catalog.columns]
missing_fields = [field for field in required_catalog_fields if field not in catalog.columns]

if missing_fields:
    print(f"Warning: Missing catalog fields: {missing_fields}")

catalog_fields = catalog[available_fields].copy()

# Get all unique months from forecast_output
all_months_sorted = sorted(forecast_output['MONTH'].unique())
month_col_names = [m.strftime('%Y-%m') for m in all_months_sorted]

# Create pivot tables for UNIT SALES and UNIT DEMAND
unit_sales_pivot = forecast_output.pivot_table(
    index='SKU', 
    columns='MONTH', 
    values='units_sold', 
    aggfunc='sum',
    fill_value=0
)
# Reindex to ensure all months are present
unit_sales_pivot = unit_sales_pivot.reindex(columns=all_months_sorted, fill_value=0)
unit_sales_pivot = unit_sales_pivot.reset_index()
unit_sales_pivot.columns = ['SKU'] + month_col_names

unit_demand_pivot = forecast_output.pivot_table(
    index='SKU', 
    columns='MONTH', 
    values='unit_demand', 
    aggfunc='sum',
    fill_value=0
)
# Reindex to ensure all months are present
unit_demand_pivot = unit_demand_pivot.reindex(columns=all_months_sorted, fill_value=0)
unit_demand_pivot = unit_demand_pivot.reset_index()
unit_demand_pivot.columns = ['SKU'] + month_col_names

# Round all numeric columns to nearest whole number
for col in month_col_names:
    unit_sales_pivot[col] = unit_sales_pivot[col].round(0).astype(int)
    unit_demand_pivot[col] = unit_demand_pivot[col].round(0).astype(int)

# Add METRIC column
unit_sales_pivot['METRIC'] = 'Unit Sales'
unit_demand_pivot['METRIC'] = 'Unit Demand'

# Combine both metrics
combined_pivot = pd.concat([unit_sales_pivot, unit_demand_pivot], ignore_index=True)

# Merge with catalog fields
combined_pivot = combined_pivot.merge(catalog_fields, on='SKU', how='left')

# Reorder columns: SKU, METRIC, catalog fields (except SKU), then months
catalog_cols_no_sku = [col for col in available_fields if col != 'SKU']
final_cols = ['SKU', 'METRIC'] + catalog_cols_no_sku + month_col_names
combined_pivot = combined_pivot[final_cols]

# Sort by SKU and METRIC (Unit Demand first, then Unit Sales for each SKU)
combined_pivot['METRIC_ORDER'] = combined_pivot['METRIC'].map({'Unit Demand': 0, 'Unit Sales': 1})
combined_pivot = combined_pivot.sort_values(['SKU', 'METRIC_ORDER']).reset_index(drop=True)
combined_pivot = combined_pivot.drop(columns=['METRIC_ORDER'])

# Fill NaN values with 0 for numeric columns and empty string for text columns
for col in month_col_names:
    combined_pivot[col] = combined_pivot[col].fillna(0)

# Export to Excel with formatting (with nan_inf_to_errors option)
with pd.ExcelWriter(demand_forecast_pivot_file, engine='xlsxwriter', engine_kwargs={'options': {'nan_inf_to_errors': True}}) as writer:
    combined_pivot.to_excel(writer, sheet_name='Demand Forecast', index=False)
    
    workbook = writer.book
    worksheet = writer.sheets['Demand Forecast']
    
    # Header format
    header_format = workbook.add_format({
        'bold': True,
        'bg_color': '#4472C4',
        'font_color': 'white',
        'border': 1,
        'align': 'center',
        'valign': 'vcenter'
    })
    
    # Number format (whole numbers)
    number_format = workbook.add_format({
        'num_format': '#,##0',
        'border': 1,
        'align': 'right'
    })
    
    # Text format
    text_format = workbook.add_format({
        'border': 1,
        'align': 'left'
    })
    
    # Unit Demand row format (light blue background)
    demand_format = workbook.add_format({
        'bg_color': '#D9E1F2',
        'border': 1,
        'align': 'left'
    })
    
    demand_number_format = workbook.add_format({
        'bg_color': '#D9E1F2',
        'num_format': '#,##0',
        'border': 1,
        'align': 'right'
    })
    
    # Apply header format
    for col_num, value in enumerate(combined_pivot.columns.values):
        worksheet.write(0, col_num, value, header_format)
    
    # Apply data formatting
    for row_num in range(1, len(combined_pivot) + 1):
        is_demand_row = combined_pivot.iloc[row_num - 1]['METRIC'] == 'Unit Demand'
        
        for col_num, col_name in enumerate(combined_pivot.columns):
            value = combined_pivot.iloc[row_num - 1, col_num]
            
            # Handle NaN values
            if pd.isna(value):
                value = 0 if col_name in month_col_names else ''
            
            # Check if it's a month column (numeric data)
            is_month_col = col_name in month_col_names
            
            if is_demand_row:
                if is_month_col:
                    worksheet.write(row_num, col_num, value, demand_number_format)
                else:
                    worksheet.write(row_num, col_num, value, demand_format)
            else:
                if is_month_col:
                    worksheet.write(row_num, col_num, value, number_format)
                else:
                    worksheet.write(row_num, col_num, value, text_format)
    
    # Set column widths
    worksheet.set_column(0, 0, 25)  # SKU
    worksheet.set_column(1, 1, 12)  # METRIC
    worksheet.set_column(2, 2, 40)  # SKU_DESCRIPTION
    worksheet.set_column(3, len(catalog_cols_no_sku) + 1, 15)  # Catalog fields
    worksheet.set_column(len(catalog_cols_no_sku) + 2, len(combined_pivot.columns) - 1, 10)  # Month columns
    
    # Freeze first row and first two columns
    worksheet.freeze_panes(1, 2)

print(f"\n✓ Created demand_forecast_pivot: {demand_forecast_pivot_file}")
print(f"  Rows: {len(combined_pivot):,} (2 metrics per SKU)")
print(f"  SKUs: {combined_pivot['SKU'].nunique():,}")
print(f"  Metrics: Unit Demand, Unit Sales")
print(f"  Month columns: {len(month_col_names)}")
print(f"  All numbers rounded to nearest whole number")

print(f"\n{'='*60}")
print("All output files created successfully!")
print(f"{'='*60}")

✓ Created demand_forecast_file: C:\Users\JaeheeKim\demand-forecast-hackathon-1\notebooks\..\data\demand_forecast_file.csv
  Rows: 293,232

✓ Created demand_forecast_sku: C:\Users\JaeheeKim\demand-forecast-hackathon-1\notebooks\..\data\demand_forecast_sku.csv
  Unique SKUs: 6,109



✓ Created demand_forecast_pivot: C:\Users\JaeheeKim\demand-forecast-hackathon-1\notebooks\..\data\demand_forecast_pivot.xlsx
  Rows: 12,218 (2 metrics per SKU)
  SKUs: 6,109
  Metrics: Unit Demand, Unit Sales
  Month columns: 48
  All numbers rounded to nearest whole number

All output files created successfully!
